# Test Code for Heat Map

In [2]:
import os

import pandas as pd
import numpy as np
import requests

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

from heat_api_keys import arcos_api_key

app = Flask(__name__)

## Arcos API Request to build Latitude and Longitude Data

In [3]:
base_url = 'https://arcos-api.ext.nile.works/v1/'
latlon_sub_url = 'pharmacy_latlon?'
state = ""
county = ""
latlon_url = base_url + latlon_sub_url + 'state=' + state + '&county=' + county + '&key=' + arcos_api_key

# Use API request to get Lat/Lon data
latlon_list = (requests.get(latlon_url)).json()
latlon_df = pd.DataFrame(latlon_list) 

pop_sub_url = 'county_population?'
pop_url = base_url + pop_sub_url + 'state=' + state + '&county=' + county + '&key=' + arcos_api_key
# Use API request to get population data by County
pop_list = (requests.get(pop_url)).json()
pop_df = pd.DataFrame(pop_list)


sell_sub_url = 'combined_county_annual?'
sell_url = base_url + sell_sub_url + 'state=' + state + '&county=' + county + '&key=' + arcos_api_key
# Use API request to get Opioid Sell data by County
sell_list = (requests.get(sell_url)).json()
sell_df = pd.DataFrame(sell_list)



In [4]:
#Merge Sell data with population data
pop_sell_df = pd.merge(pop_df, sell_df, how='inner', on=['BUYER_STATE','BUYER_COUNTY'])
pop_sell_df = pop_sell_df[['BUYER_STATE', 'BUYER_COUNTY', 'population', 'DOSAGE_UNIT', 'year_y']]
pop_sell_df.head()

,BUYER_STATE,BUYER_COUNTY,population,DOSAGE_UNIT,year_y
0,AL,AUTAUGA,51328.0,2277140.0,2006
1,AL,AUTAUGA,51328.0,2595590.0,2007
2,AL,AUTAUGA,51328.0,2817500.0,2008
3,AL,AUTAUGA,51328.0,3120850.0,2009
4,AL,AUTAUGA,51328.0,3174140.0,2010


In [5]:
#Merge the sell, population data with lat/lon data 
pop_sell_latlon_df = pd.merge(latlon_df, pop_sell_df, how='left', on=['BUYER_STATE','BUYER_COUNTY'])

# only interested in the latest year of the data, which is year 2012
pop_sell_latlon_df = pop_sell_latlon_df.loc[pop_sell_latlon_df['year_y'] == 2012]

# Calcualte the Per Capita Dosage 
pop_sell_latlon_df['Per Capita Dosage'] = pop_sell_latlon_df['DOSAGE_UNIT'] / pop_sell_latlon_df['population']

#Caculate the addiction index, which is Per Capita Dosage / number of days in a year 
pop_sell_latlon_df['Addiction_Index'] = pop_sell_latlon_df['Per Capita Dosage'] / 365

pop_sell_latlon_df = pop_sell_latlon_df.reset_index()

pop_sell_latlon_df.head()

,index,BUYER_COUNTY,BUYER_DEA_NO,BUYER_STATE,lat,lon,population,DOSAGE_UNIT,year_y,Per Capita Dosage,Addiction_Index
0,6,BALDWIN,BR5156078,AL,30.8804,-87.7845,168121.0,14396713.0,2012.0,85.633044,0.234611
1,13,BALDWIN,BR5156078,AL,30.8804,-87.7845,172404.0,14396713.0,2012.0,83.505679,0.228783
2,20,BALDWIN,BR5156078,AL,30.8804,-87.7845,175827.0,14396713.0,2012.0,81.879990,0.224329
3,27,BALDWIN,BR5156078,AL,30.8804,-87.7845,171997.0,14396713.0,2012.0,83.703280,0.229324
4,34,BALDWIN,BR5156078,AL,30.8804,-87.7845,175791.0,14396713.0,2012.0,81.896758,0.224375


In [48]:
pop_sell_latlon_df['state_county'] = pop_sell_latlon_df['BUYER_STATE'] + '_' + pop_sell_latlon_df['BUYER_COUNTY'] 
heat_df = pop_sell_latlon_df[['lat', 'lon', 'state_county', 'Per Capita Dosage', 'Addiction_Index']]
heat_df.rename(columns={'Per Capita Dosage':'per_capita_usage', 'Addiction_Index':'addiction_index'}, inplace=True)
heat_df = heat_df.loc[heat_df['addiction_index'] >= 0.4]
heat_df.head()

,lat,lon,state_county,per_capita_usage,addiction_index
1694,33.7144,-87.3883,AL_WALKER,159.811091,0.437839
1695,33.7144,-87.3883,AL_WALKER,161.083536,0.441325
1696,33.7144,-87.3883,AL_WALKER,160.751087,0.440414
1697,33.7144,-87.3883,AL_WALKER,158.312282,0.433732
1698,33.7144,-87.3883,AL_WALKER,161.281281,0.441867


In [50]:
engine = create_engine('sqlite:///db/heat.sqlite', echo=False)

heat_list_df.to_sql('HeatTable', con=engine, index=False, if_exists='append')


In [55]:
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db/heat.sqlite'
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(db.engine, reflect=True)

# save references to each table
HeatTable = Base.classes.HeatTable

C:\Users\rinku\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:835: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [57]:
heatQuery = db.session.query(HeatTable).statement
my_df = pd.read_sql_query(heatQuery, db.session.bind)


my_list = {
    "len": len(my_df),
    "lat": my_df.lat.values.tolist(),
    "lon": my_df.lon.values.tolist(),
    "state_county": my_df.state_county.values.tolist(),
    "per_capita_usage": my_df.per_capita_usage.values.tolist(),
    "addiction_index": my_df.addiction_index.values.tolist()
}

print(my_list['lat'][0])

33.7144
